In [177]:
import pandas as pd
import numpy as np 
from util import get_split_cols, get_split_frame
from sklearn.preprocessing import StandardScaler

In [178]:
train = pd.read_csv("/data/project2/train.csv")

test = pd.read_csv("/data/project2/test.csv")

In [179]:
num_train_cols, cat_train_cols = get_split_cols(train)
num_test_cols, cat_test_cols = get_split_cols(test)

In [180]:
assert(len(num_train_cols)==len(num_test_cols))
assert(len(cat_train_cols)==len(cat_test_cols))


In [181]:
num_train_df, cat_train_df = get_split_frame(train)
num_test_df, cat_test_df = get_split_frame(test)

In [182]:
y = num_train_df['Total Household Income']


In [684]:
class Layer(object):
    def __init__(self, nodes, prev_nodes, eta, activation='relu'):
        self.output = np.ones(nodes)
        self.weights = np.random.rand(nodes, prev_nodes)
        #print(self.weights)
        self.deltas = np.ones(nodes)
        self.activation_type = activation
        self.eta = eta
        
    def forward(self, inputs, no_act=False):
        self.inputs = inputs
        if no_act:
            self.output = self.activation(inputs, True)
        else:
            self.output = self.activation(inputs)
#         print("inputs", self.inputs)
#         print("weights ", self.weights)
#         print("output ", self.output)
        
    def activation(self, inputs, no_act=False):
        i_w = np.dot(inputs, self.weights.T)
        self.i_w = i_w 
        if no_act:
            return i_w
        if self.activation_type == 'sigmoid':
            return 1 / (1 + np.exp(-i_w))
        elif self.activation_type == 'relu':
            return np.maximum(i_w, 0)
        elif self.activation == 'tanh': #doesn't work 
            return np.tanh(i_w)
    
    def d_activation(self, inputs):
        inputs = np.array(inputs)
        if self.activation_type == 'sigmoid':
            return inputs - inputs ** 2
        elif self.activation_type == 'relu':
            return np.where(np.array(inputs) > 0, 1, 0)
        elif self.activation == 'tanh':
            return 1-inputs **2

    def layer_outputs(self):
        print(self.weights)
        print(self.output)
        
    def backward(self, next_deltas, output=False):

#         print("next deltas", next_deltas)
        #self.deltas = np.matrix(self.d_activation(self.i_w)).T * np.matrix(next_deltas)
        
        #WORKS but needs to include the d_activation too 
        

#         print("self deltas", self.deltas)
#         print("input", self.inputs)
#         print("weights", self.weights)
        if output:
#             print("self * next d", next_deltas * self.inputs)
            self.deltas = np.matrix(self.weights).T  * np.matrix(next_deltas) 
            self.weights -= self.eta * self.inputs * next_deltas 
        else:
            self.deltas = np.matrix(self.weights).T  * np.matrix(next_deltas) 
            self.deltas = self.deltas * self.d_activation(self.inputs)
            self.weights -= self.eta * self.d_activation(self.i_w) * self.inputs * next_deltas

            #WORKS (but wrong)
            #self.weights -= self.eta  * next_deltas.T * self.weights
        
#         print("self weights updated", self.weights)

class NeuralNetwork(object):
    
    def __init__(self, num_layers, num_nodes_per_layer, num_inputs, num_outputs, eta=.005, act='relu'):
        assert(num_layers == len(num_nodes_per_layer))
        num_nodes_per_layer = [num_inputs] + num_nodes_per_layer + [num_outputs]
        self.layers = [Layer(num_nodes_per_layer[i], num_nodes_per_layer[i-1], eta, act) for i in range(1, num_layers+2)]
        
    def forward(self, x):
        for layer in self.layers[:-1]:
            layer.forward(x)
            x = layer.output
        self.layers[-1].forward(x, True)
            
    def backward(self, expected):
        deltas = self.layers[-1].output - expected
        #print(deltas)
        self.layers[-1].backward(deltas, True)
        #self.layer[-1].deltas = np.matrix(deltas) * self.layer[-2].output 
        deltas = self.layers[-1].deltas
        
        for layer in reversed(self.layers[:-1]):
            layer.backward(deltas)
            deltas = layer.deltas
            
            
    def train(self, x, y, epoch):
        for i in range(epoch):
            self.sum_error = 0 
#             print(i)
            for xi, yi in zip(x,y):
                #print(xi, yi)
                self.forward(xi)
                self.backward(yi)
                self.sum_error += (self.layers[-1].output - yi)**2
            print(self.sum_error)
            
            
    def predict(self, x):
        self.forward(x)
        return self.layers[-1].output

In [685]:
nn = NeuralNetwork(1, [ 2], 2, 1, eta=.00005, act='relu')

In [686]:
data = [[1, 1],[2, 2],[3, 3],[4, 4],[5, 5],[6, 6],[7, 7],[8, 8],[9, 9],[10, 10],[11, 11],[12, 12],[13, 13],[14, 14],[15, 15],[16, 16],[17, 17],[18, 18],[19, 19],[20, 20],[21, 21],[22, 22],[23, 23],[24, 24],[25, 25],[26, 26],[27, 27],[28, 28],[29, 29],[30, 30],[31, 31],[32, 32],[33, 33],[34, 34],[35, 35],[36, 36],[37, 37],[38, 38],[39, 39],[40, 40],[41, 41],[42, 42],[43, 43],[44, 44],[45, 45],[46, 46],[47, 47],[48, 48],[49, 49],[50, 50],[51, 51],[52, 52],[53, 53],[54, 54],[55, 55],[56, 56],[57, 57],[58, 58],[59, 59],[60, 60],[61, 61],[62, 62],[63, 63],[64, 64],[65, 65],[66, 66],[67, 67],[68, 68],[69, 69],[70, 70],[71, 71],[72, 72],[73, 73],[74, 74],[75, 75],[76, 76],[77, 77],[78, 78],[79, 79],[80, 80],[81, 81],[82, 82],[83, 83],[84, 84],[85, 85],[86, 86],[87, 87],[88, 88],[89, 89],[90, 90],[91, 91],[92, 92],[93, 93],[94, 94],[95, 95],[96, 96],[97, 97],[98, 98],[99, 99],[100, 100]]
expected = [2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200]

In [687]:
nn.train(data, expected, 10)
nn.predict([2,2])

[ 9026.23792568]
[  8.41255306e-11]
[  3.93737920e-11]
[  1.97122877e-10]
[  1.83492620e-10]
[  4.66998763e-11]
[  5.11542303e-11]
[  1.41879930e-11]
[  1.01242189e-11]
[  2.95088008e-12]


array([ 4.00000009])

In [668]:
nn.train([[2,1]], [3], 1)
nn.predict([2,2])

[ 0.00665416]


array([ 3.99989232])

In [669]:
scalar_x = StandardScaler()
scalar_y = StandardScaler()

x = scalar_x.fit_transform(num_train_df.values)
yy = scalar_y.fit_transform(y.values.reshape(-1,1))
#yy =  y/1000000.0
inputs = len(num_train_df.columns)

num_nodes = [inputs] * 1 

n = NeuralNetwork(num_layers = 1, num_nodes_per_layer= num_nodes, num_inputs=inputs,
                  num_outputs=1, eta=.0005, act='relu')
n.train(x, yy, 10)

/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[ 36891.29872442]
[ 8985.388649]
[ 8980.68128425]
[ 8979.73709472]
[ 8979.68468991]
[ 8979.68387275]
[ 8979.6838226]
[ 8979.68381973]
[ 8979.68381957]
[ 8979.68381956]


In [622]:
print(scalar_y.inverse_transform([7722.55]))
print(scalar_y.inverse_transform(n.predict(x[0])))
#print(n.predict(x[0]) * 1000000)
print(y[0])


[  2.21591933e+09]
[ 575699.98325547]
115835
